In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [2]:
# Parameters
N = 100
L = 10.0
T = 1000
dt = 0.05
k = 10.0  
r0 = 1.0        
rc = 2.0  

plt.rcParams['animation.embed_limit'] = 1000

mass = np.ones((N, 1))
radius = np.full((N, 1), 0.1)

pos = np.random.uniform(0, L, size=(N, 3))
vel = np.random.uniform(-1, 1, size=(N, 3))

In [3]:
def walls_reflect(pos, vel, L, x, y,z):


    vel[x, 0] *= -1
    pos[x, 0] = np.where(pos[x, 0] < 0, 0.0, np.where(pos[x, 0] > L, L, pos[x, 0]))


    vel[y, 1] *= -1
    pos[y, 1] = np.where(pos[y, 1] < 0, 0.0, np.where(pos[y, 1] > L, L, pos[y, 1]))

    vel[z, 2] *= -1
    pos[z, 2] = np.where(pos[z, 2] < 0, 0.0, np.where(pos[z, 2] > L, L, pos[z, 2]))

In [4]:
def pair_forces(pos, L):

    d = pos[None, :, :] - pos[:, None, :]
    d[:, :, 0] = d[:, :, 0] - L * np.round(d[:, :, 0] / L)
    d[:, :, 1] = d[:, :, 1] - L * np.round(d[:, :, 1] / L)
    r = np.linalg.norm(d, axis=2)
    mask = (r < rc) & (r > 1e-12)
    u = d / (r[:, :, None] + 1e-12)
    fmag = -1 * k * (r - r0) * mask
    F = np.sum(fmag[:, :, None]*u, axis=1)
    return F

In [5]:
def step_once(pos,vel,dt,L):
    forces = pair_forces(pos,L)
    acc = forces / mass
    vel += acc * dt
    pos += vel * dt

    x = (pos[:, 0] < 0) | (pos[:, 0] > L)
    y = (pos[:, 1] < 0) | (pos[:, 1] > L)
    z = (pos[:, 2] < 0) | (pos[:, 2] > L)
    walls_reflect(pos, vel, L, x, y,z)

    return pos,vel

In [6]:
def detect_collisions(pos, radius):
    d = pos[None, :, :] - pos[:, None, :]
    r = np.linalg.norm(d, axis=2)
    colliding_pairs = np.array(np.where((r < 2 * radius) & (r > 1e-12))).T
    colliding_pairs = colliding_pairs[colliding_pairs[:,0] < colliding_pairs[:,1]]
    return colliding_pairs

In [7]:
def merge_particles(pos, vel, mass, radius, i, j):
    radius[i, 0] = np.sqrt(radius[i, 0]**2 + radius[j, 0]**2)
    total_mass = mass[i, 0] + mass[j, 0]
    vel[i] = (mass[i, 0]*vel[i] + mass[j, 0]*vel[j]) / total_mass
    mass[i, 0] = total_mass
    pos[i] = (pos[i] + pos[j]) / 2

    pos = np.delete(pos, j, axis=0)
    vel = np.delete(vel, j, axis=0)
    mass = np.delete(mass, j, axis=0)
    radius = np.delete(radius, j, axis=0)
    return pos, vel, mass, radius

In [8]:
def kinetic_energy(mass, vel):
    v2 = np.sum(vel**2, axis=1)
    return 0.5 * np.sum(mass.flatten() * v2)

def potential_energy(pos, k, r0, rc):
    d = pos[None, :, :] - pos[:, None, :]
    r = np.linalg.norm(d, axis=2)
    mask = (r < rc) & (r > 1e-12)
    return -0.5 * np.sum(0.5 * k * (r[mask] - r0)**2)


In [ ]:

plt.close('all')
fig = plt.figure(figsize=(12, 5), dpi=150)
ax = fig.add_subplot(121, projection='3d')
ax_energy = fig.add_subplot(122)
plt.subplots_adjust(wspace=0.4)

ax.set_xlim(0, L)
ax.set_ylim(0, L)
ax.set_zlim(0, L)
ax.set_title("Spring-Connected Particles (3D)")
ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')

scatter = ax.scatter(pos[:, 0], pos[:, 1], pos[:, 2], s=(radius.flatten()*500)**2, c='tab:blue')

ax_energy.set_title("Energy vs Time")
ax_energy.set_xlabel("Time (s)")
ax_energy.set_ylabel("Energy")
ke_line, = ax_energy.plot([], [], 'r-', label="Kinetic")
pe_line, = ax_energy.plot([], [], 'b-', label="Potential")
te_line, = ax_energy.plot([], [], 'g-', label="Total")
ax_energy.legend()

times, KE_values, PE_values, TE_values = [], [], [], []

def init():
    scatter._offsets3d = (pos[:, 0], pos[:, 1], pos[:, 2])
    scatter.set_sizes((radius.flatten() * 69) ** 2)
    ke_line.set_data([], [])
    pe_line.set_data([], [])
    te_line.set_data([], [])
    times.clear()
    KE_values.clear()
    PE_values.clear()
    TE_values.clear()
    return scatter, ke_line, pe_line, te_line

delta_ke = ax_energy.text(0.02, 0.95, '', transform=ax_energy.transAxes, fontsize=12, color='blue')


ke_max_ss = 0.0
ke_sum_ss = 0.0
ke_count_ss = 0

def animate(frame):
    global pos, vel, mass, radius, ke_max_ss, ke_sum_ss, ke_count_ss
    pos, vel = step_once(pos, vel, dt,L)
    while True:
        pairs = detect_collisions(pos, radius)
        if len(pairs) == 0:
            break
        i, j = pairs[0]
        pos, vel, mass, radius = merge_particles(pos, vel, mass, radius, i, j)
    
    scatter._offsets3d = (pos[:, 0], pos[:, 1], pos[:, 2])
    scatter.set_sizes((radius.flatten() * 69) ** 2) 
    t = frame * dt
    KE = kinetic_energy(mass, vel)
    PE = potential_energy(pos, k, r0, rc)
    TE = KE + PE
    
    times.append(t)
    KE_values.append(KE)
    PE_values.append(PE)
    TE_values.append(TE)
    
    ke_line.set_data(times, KE_values)
    pe_line.set_data(times, PE_values)
    te_line.set_data(times, TE_values)

    ax_energy.set_xlim(0, max(0.1, t + dt))
    all_energies = KE_values + PE_values if (len(KE_values) > 0 and len(PE_values) > 0) else (KE_values or PE_values or [0.0])
    ymin = min(all_energies) * 1.2
    ymax = max(all_energies) * 1.2
    if ymin == ymax:
        ymin -= 1.0
        ymax += 1.0
    ax_energy.set_ylim(ymin, ymax)

    if len(TE_values) > 1 and TE_values[-2] != 0:
        delta_te_percent = 100 * abs(TE_values[-1] - TE_values[-2]) / abs(TE_values[-2])
        ke_max_ss = max(ke_max_ss, delta_te_percent)
        ke_sum_ss += delta_te_percent
        ke_count_ss += 1
        ke_mean = ke_sum_ss / ke_count_ss if ke_count_ss > 0 else 0.0
        delta_ke.set_text(f"STD: TE {np.std(TE_values):.1f}  STD: KE {np.std(KE_values):.1f}  STD: PE {np.std(PE_values):.1f}")
    else:
        delta_ke.set_text("ΔTE%: 0.00%")
    
    return scatter, ke_line, pe_line, te_line

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=300, interval=10, blit=False)


# with open("3D_MS.html", "w") as f:
#     f.write(anim.to_jshtml())
HTML(anim.to_jshtml())